In [96]:
import numpy as np 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import control.matlab as control_
import control
import math 
from sympy import symbols, solve

def pos(lst):
    return [x for x in lst if x > 0] or None

def jitter(inps):
    ICP,PM,FC= inps
    pi=math.pi
    X= symbols('x')
    PM=math.radians(PM)
    eq=((X-(1/X))/2)-(np.tan(PM))
    x=float(pos(solve(eq,X))[0])
    #Assigning some constants 
    Fref=320e6
    Fc=(FC)
    Icp=(ICP)
    R_ratio=2
    P_divider=6
    N=21
    Fo=560e6
    KVCO=160e6
    Icp=ICP
    Fp=x*Fc
    Fz=Fc/x
    k=(KVCO*Icp)/(2*pi*N)
    R1=(2*(pi)*Fc)/(k*(1-(1/(x**2))))
    wz=2*pi*Fz                       
    C1=1/(R1*wz)
    C2=C1/((x**2)-1)
    Ceq=C1*C2/(C1+C2)
    print(Ceq,C1,C2,R1)
    #VCO phase noise 
    w=np.linspace(2*pi*12e3,2*pi*20e6,50000)
    T_osc=control.tf([1,315.234e3],[1,0])
    T_osc=T_osc*104.5
    s=control.tf([1],[1,0,0])
    T_osc=T_osc*s
    T_osc

    #VCO phase noise 
    (mag, phase_rad, w) = control.bode(T_osc, w,dB=True,Hz=True,plot=False)
    for i in range(np.shape(w)[0]):
        if 10*np.log10(mag[i]) <= -145 :
            mag[i:]=mag[i]
            break
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(mag), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('VCO phase noise')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    #Reference phase noise 
    T_ref=control.tf([1,16000*pi],[1,0])
    T_ref=T_ref*3.8115e-3
    s=control.tf([1],[1,0,0])
    T_ref=T_ref*s
    T_ref

    #Reference phase noise 
    (mag_ref, phase_rad_ref, w) = control.bode(T_ref, w,dB=True,Hz=True,plot=False)
    for i in range(np.shape(w)[0]):
        if 10*np.log10(mag_ref[i]) <= -150 :
            mag_ref[i:]=mag_ref[i]
            break
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(mag_ref), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('Reference phase noise')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    # transfer noise of loop filter is RC circuit noise 
    T_loopfilter=4*4.11e-21*R1

    #CP 
    T_CP_noise=control.tf([1,2*2e6*pi],[1,0])
    T_CP_noise=T_CP_noise*(5.10613e-24*(Icp))/(300e-6)
    (mag_CP_noise, phase_rad_CP_noise, w) = control.bode(T_CP_noise, w,dB=True,Hz=True,plot=False)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(mag_CP_noise), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('CP phase noise')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    # Feedforward for Feedback loop 
    Ts=control.tf([R1*C1*k,k],[R1*Ceq,1,0,0])
    Ts=Ts/((C1+C2))
    (mag_Feed, phase_rad_Feed, w) = control.bode_plot(Ts, w,dB=True, deg= True , margins=True,plot=False)


    # Feedback of the loop in the middle 
    #Closed loop 
    Bs=(Ts)/(1+(Ts))
    (mag_FB, phase_rad_FB, w) = control.bode_plot(Bs, w,dB=True, deg= True,plot=False)

    # the overall closed loop
    Bs=(Ts*N)/((1+(Ts))*(R_ratio*P_divider))
    (mag_FB, phase_rad_FB, w) = control.bode_plot(Bs, w,dB=True, deg= True,plot=False)

    #Transfer function of different blocks 
    #Transfer function of VCO phase output to phase of VCO
    T_out_VCO=1/((1+(Ts))*P_divider)
    (mag_out_VCO, phase_out_VCO, w) = control.bode_plot(T_out_VCO, w,dB=True, deg= True,plot=False)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10((mag_out_VCO)**2), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('(phase noise/ output)^2')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    VCO_noise_output=mag*((mag_out_VCO)**2)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(VCO_noise_output), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('VCO noise at output')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    #Transfer function of different blocks 
    #Transfer function of CP phase output to phase of CP
    T_out_CP=Ts*((2*pi)/Icp)*N
    T_out_CP=T_out_CP/(1+Ts)
    T_out_CP=T_out_CP/P_divider
    (mag_out_CP, phase_out_CP, w) = control.bode_plot(T_out_CP, w,dB=True, deg= True,plot=False)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10((mag_out_CP)**2), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('(CP phase noise/ output)^2')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    CP_noise_output=mag_CP_noise*((mag_out_CP)**2)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(CP_noise_output), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('CP noise at output')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    #Transfer function of different blocks 
    #Transfer function of CP phase output to phase of Reference
    T_out_ref=Bs
    (mag_out_ref, phase_out_ref, w) = control.bode_plot(T_out_ref, w,dB=True, deg= True,plot=False)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10((mag_out_ref)**2), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('(Ref phase noise/ output)^2')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    ref_noise_output=mag_ref*((mag_out_ref)**2)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(ref_noise_output), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('ref noise at output')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    #Transfer function of different blocks 
    #Transfer function of LOOP_FILTER phase output to phase of Reference

    s_=control.tf([1,0],[1])
    T_out_loop_filter=((KVCO/s_)*(1/P_divider))/(1+Ts)
    (mag_out_loop_filter, phase_out_loop_filter, w) = control.bode_plot(T_out_loop_filter, w,dB=True, deg= True,plot=False)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10((mag_out_loop_filter)**2), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('loop_filter phase noise/ output)^2')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    loop_filter_noise_output=(4.112e-21)*4*R1*((mag_out_loop_filter)**2)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(loop_filter_noise_output), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('loop Filter noise at output')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    #Transfer function of R_Divider phase output to phase of Reference

    T_out_R_Divider=Bs*R_ratio
    (mag_out_R_Divider, phase_out_R_Divider, w) = control.bode_plot(T_out_R_Divider, w,dB=True, deg= True,plot=False)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10((mag_out_R_Divider)**2), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('R_Divider phase noise/ output)^2')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    # Divider transfer phase noise 
    T_Divider=control.tf([1,2*2e6*pi],[1,0])
    T_Divider=T_Divider*7.0710678e-17

    (mag_Divider, phase_rad_Divider, w) = control.bode(T_Divider, w,dB=True,Hz=True,plot=False)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(mag_Divider), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('Dividers phase noise')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    R_divider_noise_output=((mag_out_R_Divider)**2)*mag_Divider
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(R_divider_noise_output), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('R Divider noise at output')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    #Transfer function of N_Divider phase output to phase of Reference

    T_out_N_Divider=Bs*R_ratio
    (mag_out_N_Divider, phase_out_N_Divider, w) = control.bode_plot(T_out_N_Divider, w,dB=True, deg= True,plot=False)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10((mag_out_N_Divider)**2), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('N_Divider phase noise/ output)^2')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    N_divider_noise_output=((mag_out_N_Divider)**2)*mag_Divider
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(N_divider_noise_output), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('N Divider noise at output')
#     plt.grid() 
#     plt.legend(labels=('mag',))


#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(2*mag_Divider), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('P Divider and Buffer noise at output')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    #calculation every noise at the output 
    noise_output=VCO_noise_output+CP_noise_output+ref_noise_output+loop_filter_noise_output+R_divider_noise_output+N_divider_noise_output+(2*mag_Divider)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(noise_output), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('noise at output')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    from scipy import integrate
    jitter= np.trapz(noise_output, x=w/(2*pi))
    jitter=jitter*2
    jitter=(math.sqrt(jitter))/(2*pi*Fo)
    return(jitter*1e15)

In [97]:
jitter([1.00000000e-03, 7.50000000e+01, 2.33858677e+06])

7.265907325591294e-13 4.192100147162246e-11 7.394064054421545e-13 12331.215612040596


194.16488953369694

In [98]:
import scipy.optimize as spo
bnds=((10e-6,1e-3),(60,75),(1e3,16e6))
intials=[1e-3,75,2.33858677e+06]
res=spo.minimize(jitter,intials,method='Nelder-Mead',bounds=bnds)

7.265907325591294e-13 4.192100147162246e-11 7.394064054421545e-13 12331.215612040596
7.265907325591294e-13 4.192100147162246e-11 7.394064054421545e-13 12331.215612040596
7.265907325591294e-13 4.192100147162246e-11 7.394064054421545e-13 12331.215612040596
6.590392132055595e-13 3.802358410124486e-11 6.706634062967387e-13 12947.776392642627
8.050866842760433e-13 4.6449863126451466e-11 8.19286875836182e-13 11714.654831438567
6.915795193900101e-13 3.99010127035074e-11 7.037776613369703e-13 12639.496002341612
7.643295017058558e-13 4.4098357892567996e-11 7.778108144033181e-13 12022.935221739583
7.087609645447993e-13 4.0892304438101477e-11 7.212621543712527e-13 12485.355807191103
7.451018568143607e-13 4.298900968088185e-11 7.582440305767964e-13 12177.075416890091
7.175927916945225e-13 4.140186094955964e-11 7.30249758084918e-13 12408.285709615848
7.205734439043035e-13 4.15738311111285e-11 7.332829833072451e-13 12382.595677090763
7.175927916945225e-13 4.140186094955964e-11 7.30249758084918e-13 1

7.204711387646555e-13 4.156792856693468e-11 7.331788737003154e-13 12383.47479503474
7.204712652079579e-13 4.156793586214177e-11 7.331790023738361e-13 12383.47370837966
7.204711567644964e-13 4.156792960544416e-11 7.331788920176392e-13 12383.474640343897
7.204712489636572e-13 4.156793492491905e-11 7.331789858430171e-13 12383.473847983334
7.20471187497548e-13 4.156793137860233e-11 7.33178923292763e-13 12383.47437622371
7.204712335971293e-13 4.156793403833983e-11 7.331789702054529e-13 12383.473980043429
7.204711947419248e-13 4.156793179657014e-11 7.331789306649169e-13 12383.474313965457
7.204712254440364e-13 4.156793356794322e-11 7.331789619085551e-13 12383.474050111192
7.204712173218867e-13 4.156793309933189e-11 7.331789536431462e-13 12383.47411991303
7.204711865888328e-13 4.156793132617358e-11 7.331789223680199e-13 12383.47438403322
7.204712096386231e-13 4.156793265604231e-11 7.331789458243645e-13 12383.474185943076
7.204711902110215e-13 4.156793153515749e-11 7.331789260540971e-13 12383.

In [99]:
print(res)

 final_simplex: (array([[1.00000000e-03, 7.50000000e+01, 2.34849753e+06],
       [1.00000000e-03, 7.50000000e+01, 2.34849753e+06],
       [1.00000000e-03, 7.50000000e+01, 2.34849753e+06],
       [1.00000000e-03, 7.50000000e+01, 2.34849753e+06]]), array([194.1645216, 194.1645216, 194.1645216, 194.1645216]))
           fun: 194.16452159698892
       message: 'Optimization terminated successfully.'
          nfev: 158
           nit: 68
        status: 0
       success: True
             x: array([1.00000000e-03, 7.50000000e+01, 2.34849753e+06])
